In [41]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date

games = pd.read_csv('games_2009_2011.csv')
games.set_index('match_id')
BASE_URL = 'http://espn.go.com/nba/boxscore?gameId={0}'

request = requests.get(BASE_URL.format(games['match_id'][0]))

table = BeautifulSoup(request.text,'lxml').find('table', class_='mod-data')
heads = table.find_all('thead')
#headers = heads[0].find_all('tr')[1].find_all('th')[1:]
headers = heads[0].find_all('tr')[0].find_all('th')[1:]
headers = [th.text for th in headers]
columns = ['match_id', 'team', 'playerId'] + headers
players = pd.DataFrame(columns=columns)
players



,match_id,team,playerId,MIN,FG,3PT,FT,OREB,DREB,REB,AST,STL,BLK,TO,PF,+/-,PTS


In [42]:
def get_players(players, team_name, matchId):
    array = np.zeros((len(players), len(headers)+1), dtype=object)
    array[:] = np.nan
    for i, player in enumerate(players):
        cols = player.find_all('td')
        if cols[1].text.startswith('DNP') or cols[1].text.startswith('NWT'):
            continue
        try:
            array[i, 0] = cols[0].a['href'].split('/')[7]
        except:
            pass
        for j in range(1, len(headers) + 1):
            array[i, j] = cols[j].text
    frame = pd.DataFrame(columns=columns)
    for x in array:
        line = np.concatenate(([matchId, team_name], x)).reshape(1,len(columns))
        new = pd.DataFrame(line, columns=frame.columns)
        frame = frame.append(new)
    return frame



In [45]:
for index, row in games.iterrows():
    if index > 50:   #for test
        break
    print((index), row['match_id'])
    request = requests.get(BASE_URL.format(row['match_id']))
    bodies = BeautifulSoup(request.text,'lxml').find_all('tbody')
    team_1 = 'visit'
    try:
        team_1_players = bodies[1].find_all('tr') + bodies[2].find_all('tr')[:-2]
        team_1_players = get_players(team_1_players, team_1, row['match_id'])
        players = players.append(team_1_players)
        team_2 = 'home'
        team_2_players = bodies[3].find_all('tr') + bodies[4].find_all('tr')[:-2]
        team_2_players = get_players(team_2_players, team_2, row['match_id'])
        players = players.append(team_2_players)
    except:
        pass


players = players.set_index(['match_id'])

players.FG = '\'' + players.FG
players['3PT'] = '\'' + players['3PT']
players.FT = '\'' + players.FT
print(players)
#players.to_csv('GameData_2009_2011.csv')


0 281028002
1 281031002
2 281101011
3 281104010
4 281105025
5 281107002
6 281109008
7 281110002
8 281112002
9 281114002
10 281115015
11 281118002
12 281120002
13 281121016
14 281123028
15 281126002
16 281128002
17 281129030
18 281201002
19 281203002
20 281205002
21 281207011
22 281211027
23 281212002
24 281215002
25 281217001
26 281219002
27 281221002
28 281223002
29 281225013
30 281226009
31 281228023
32 281230022
33 290102002
34 290104018
35 290106030
36 290107002
37 290109005
38 290111028
39 290112002
40 290114002
41 290117017
42 290119002
43 290121014
44 290122019
45 290125002
46 290128002
47 290130008
48 290201002
49 290203020
50 290205002
            team playerId MIN      FG   3PT      FT OREB DREB REB AST STL BLK  \
match_id                                                                        
281028002  visit      885  19    '1-1  '0-0    '0-0    0    4   4   0   0   1   
281028002  visit     1966  36   '9-21  '0-4    '4-8    1    6   7   6   2   1   
281028002  visit      3

In [46]:
players.to_csv('GameData_2009_2011.csv')

In [26]:
print('\'')

'
